A possible implementation could be as teh code below :

The `challenge_A_KITT()` function is responsible for aligning KITT's trajectory to point B and driving it to reach point B using the minimum distance. 

1. **Initial Check**: The function first checks if KITT is "almost there" (line 6). If true, it calculates the distance between KITT's current position and point B (`accuracy`) and the minimum distance from KITT's trajectory to point B (`min_distance`).

2. **Positioning at Point B**: If the accuracy is within a threshold (less than or equal to 20 cm), KITT stops and marks the mission as accomplished (lines 9-15). It prints the current position and accuracy, indicating that point B has been located.

3. **Speed Control**: If KITT is not yet at point B, it adjusts its speed depending on the battery voltage, using higher speed when the voltage is low to conserve power (lines 19-26 and 32-39).

4. **Trajectory Alignment**: If KITT is on the route to point B (line 28), the system continues moving while adjusting the angle and speed. KITT tries to steer towards point B by calculating the error between the current and desired orientation and applying a proportional-derivative (PD) control to determine the steering angle (lines 65-74).

5. **Steering Adjustments**: Based on the calculated steering angle, KITT adjusts its steering to either correct its trajectory or handle small deviations in its path (lines 76-97). The steering command is adjusted in degrees depending on how far KITT is from the correct trajectory.

6. **Boundary Handling**: If KITT approaches the grid boundaries (too close to edges), it sets a safe speed and adjusts its angle to avoid going out of bounds (lines 98-105).

7. **Final Position Update**: Once the vehicle reaches point B, or if obstacles are detected en route, the system stops and updates KITT's position. If the calculated trajectory ensures that KITT is still on course to point B (line 130), the mission proceeds as planned.

The function also manages smaller details like ensuring obstacle avoidance and optimizing movement based on current battery levels throughout the process.

In [ ]:
def challenge_A_KITT():
    
    global mission_A_accomplished, accuracy

    if kittmodel.almost_there == 1:
        # Calculate the accuracy and minimum distance to point B
        accuracy = np.sqrt((kittmodel.dist_x - B_pos[0]) ** 2 + (kittmodel.dist_y - B_pos[1]) ** 2)
        min_distance = abs(-kittmodel.slope_t1 * B_pos[0] + B_pos[1] - kittmodel.b_t1) / np.sqrt(kittmodel.slope_t1 ** 2 + 1)

        # Check if KITT has reached the required accuracy at point B
        if round(100 * accuracy) <= 20:
            print("Position B located!")
            print("Accuracy:", round(100 * accuracy, 2), "cm")
            print("x:", round(kittmodel.dist_x, 3), "y:", round(kittmodel.dist_y, 3))
            kittmodel.stop()
            kitt.stop(0)
            mission_A_accomplished = 1
            return mission_A_accomplished
        else:
            kittmodel.set_speed(158)
            if kitt.battery_voltage <= 18:
                kitt.set_speed(164)
                time.sleep(0.05)
                kitt.set_speed(158)
            else:
                kitt.set_speed(161)
                time.sleep(0.05)
                kitt.set_speed(158)

    elif kittmodel.on_route_AB == 1:
        print("Found trajectory A-B")
        kittmodel.set_speed(158)
        kitt.set_angle(150)
        
        if kitt.battery_voltage <= 18:
            kitt.set_speed(164)
            time.sleep(0.05)
            kitt.set_speed(158)
        else:
            kitt.set_speed(161)
            time.sleep(0.05)
            kitt.set_speed(158)

        # Recalculate accuracy and minimum distance
        accuracy = np.sqrt((kittmodel.dist_x - B_pos[0]) ** 2 + (kittmodel.dist_y - B_pos[1]) ** 2)
        min_distance = abs(-kittmodel.slope_t1 * B_pos[0] + B_pos[1] - kittmodel.b_t1) / np.sqrt(kittmodel.slope_t1 ** 2 + 1)

        if init_TDOA == 1:
            if round(100 * accuracy) <= 25:
                print("In range 25 cm")
                kitt.stop(0)
                kittmodel.stop()
                kitt.read_microfoon()
                kittmodel.almost_there = 1
            else:
                if round(100 * accuracy) <= 10:
                    print("Position B located!")
                    print("Accuracy:", round(100 * accuracy, 2), "cm")
                    print("x:", round(kittmodel.dist_x, 3), "y:", round(kittmodel.dist_y, 3))
                    kittmodel.stop()
                    kitt.stop(0)
                    mission_A_accomplished = 1
                    return mission_A_accomplished
                else:
                    # PID controller for steering adjustment
                    Kp = 3
                    Kd = 0.1

                    current_orientation = np.arctan2(kittmodel.d_vector_0[1], kittmodel.d_vector_0[0])
                    desired_orientation = np.arctan2(B_pos[1] - kittmodel.dist_y, B_pos[0] - kittmodel.dist_x)

                    error = desired_orientation - current_orientation
                    error = (error + np.pi) % (2 * np.pi) - np.pi
                    derivative = (error - kittmodel.prev_error) / kittmodel.delta_t
                    kittmodel.prev_error = error

                    steering_angle = Kp * error + Kd * derivative

                    # Steering command based on the calculated steering angle
                    if steering_angle < 0:
                        if abs(steering_angle) <= np.radians(1):
                            steering_command = 150
                        elif abs(steering_angle) <= np.radians(5):
                            steering_command = 185
                        elif abs(steering_angle) <= np.radians(10):
                            steering_command = 190
                        elif abs(steering_angle) <= np.radians(15):
                            steering_command = 195
                        else:
                            steering_command = 200
                    else:
                        if abs(steering_angle) <= np.radians(1):
                            steering_command = 150
                        elif abs(steering_angle) <= np.radians(5):
                            steering_command = 115
                        elif abs(steering_angle) <= np.radians(10):
                            steering_command = 110
                        elif abs(steering_angle) <= np.radians(15):
                            steering_command = 105
                        else:
                            steering_command = 100

                    # Boundary condition check
                    if (kittmodel.dist_x >= 4.6 or kittmodel.dist_y >= 4.6 or kittmodel.dist_x <= 0.2 or kittmodel.dist_y <= 0.2):
                        kitt.set_angle(150)
                        kitt.set_speed(142)
                        kitt.set_speed(139)
                        time.sleep(0.1)
                        kitt.set_speed(142)
                        time.sleep(1.4)
                        kitt.stop(1)
                        kitt.read_microfoon()
                    else:
                        kittmodel.set_angle(steering_command)
                        kitt.set_angle(steering_command)
                        if steering_command == 150:
                            kittmodel.set_speed(158)
                        else:
                            kittmodel.set_speed(160)
                        if kitt.battery_voltage <= 18.1:
                            kitt.set_speed(163)
                            time.sleep(0.1)
                            kitt.set_speed(158)
                        else:
                            kitt.set_speed(161)
                            time.sleep(0.1)
                            kitt.set_speed(158)

        # Final position check with discriminant for correct trajectory
        k = 1 + kittmodel.slope_t1 ** 2
        l = 2 * kittmodel.slope_t1 * (kittmodel.b_t1 - B_pos[1]) - 2 * B_pos[0]
        m = B_pos[0] ** 2 + (kittmodel.b_t1 - B_pos[1]) ** 2 - 0.12 ** 2

        discriminant_2A = l ** 2 - 4 * k * m

        if (discriminant_2A >= 0 and ((B_pos[0] - kittmodel.dist_x) * kittmodel.d_vector_0[0] >= 0 or
                                      (B_pos[1] - kittmodel.dist_y) * kittmodel.d_vector_0[1] >= 0)):
            kittmodel.stop()
            kitt.stop(0)
            kittmodel.on_route_AB = 1


Obstacle detection 

Simple soultion for obstacle avoidance:

The code below implements obstacle detection and avoidance for KITT to complete challenges C and D. It begins by reading data from KITT’s ultrasonic sensors, which detect the distance to obstacles around the car. The function `get_distance_update()` requests distance data and decodes it to extract the distance to obstacles on the left and right sides of the car. The data is updated with an offset of 4%, as specified in the challenge. 

If the distance to any obstacle is less than or equal to 50 cm, KITT stops. This ensures that it maintains a safe distance from obstacles, matching the requirement from the challenge to avoid collisions with a 50 cm buffer for safety. The position of the obstacle is calculated and printed for reference, allowing KITT to document its location. The car then evaluates whether it should adjust its movement depending on whether the obstacle is on the left or right side, or directly in front.

 KITT handles situations where it approaches the grid boundaries as if they were obstacles. When it detects that it's near a boundary, KITT stops, adjusts its angle, and backs away to avoid going out of bounds, following the same maneuver as it does with physical obstacles.



In [ ]:

def get_distance_update():
    
    kitt.serial.write(b'Sd\n')
    distance = kitt.serial.read_until(b'\x04')
    distance = distance.decode('utf-8')
    
    dist_left = int(distance[int(distance.find('L') + 1):int(distance.find('R') - 3)])
    dist_right = int(distance[int(distance.find('R') + 1):int(distance.find('x') - 1)])
    
    offset = 4  # %, as measured
    
    # Use the parking sensors, Obstacle Detection, Threshold of 50 cm or the boundaries of the grid
    if (kittmodel.dist_x >= 4.4 or kittmodel.dist_y >= 4.4 or kittmodel.dist_x <= 0.2 or kittmodel.dist_y <= 0.2 or dist_left <= 60 or dist_right <= 60):
        if dist_left <= 60 and dist_right <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_left / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_left / 100 + 0.21))
            print("Obstacle Detected in front of the Car at Location:", kitt.Obstacle_pos)
        
        if dist_left <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_left / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_left / 100 + 0.21))
            print("Obstacle Detected left of the Car at", kitt.Obstacle_pos)
        
        if dist_right <= 60:
            kitt.Obstacle_pos = (kittmodel.dist_x + kittmodel.d_vector_0[0] * ((dist_right / 100) * (100 - offset) + 0.21),
                                 kittmodel.dist_y + kittmodel.d_vector_0[1] * (dist_right / 100 + 0.21))
            print("Obstacle Detected right of the Car at", kitt.Obstacle_pos)
        
        kitt.stop(0)
        return 1
    else:
        return 0

obstacle_detected = get_distance_update()
kittmodel.obstacle_trajectory = kittmodel.d_vector_0

if kittmodel.obstacle_seen >= 1 and kittmodel.obstacle_seen <= 8:
    obstacle_detected = 0

# Obstacle avoidance
kittmodel.obstacle_seen += 1
kitt.set_angle(kittmodel.avoidance_turn)
kittmodel.set_angle(kittmodel.avoidance_turn)
kittmodel.shifting(158)

if kitt.battery_voltage <= 18:
    kitt.set_speed(164)
    time.sleep(0.05)
    kitt.set_speed(158)
else:
    kitt.set_speed(161)
    time.sleep(0.05)
    kitt.set_speed(158)

if obstacle_detected == 1:
    kittmodel.on_route_AB = 0
    if kitt.battery_voltage <= 18:
        kitt.set_angle(150)
        kitt.set_speed(135)
        time.sleep(0.1)
        kitt.set_speed(142)
        time.sleep(1.4)
    else:
        kitt.set_angle(150)
        kitt.set_speed(139)
        time.sleep(0.1)
        kitt.set_speed(142)
        time.sleep(1.4)
    
    kitt.stop(1)
    
    if init_TDOA == 1:
        kitt.read_microfoon()
    else:
        kittmodel.dist_x -= kittmodel.d_vector_0[0] * 0.5
        kittmodel.dist_y -= kittmodel.d_vector_0[1] * 0.5
        kittmodel.obstacle_seen += 1

product1 = kittmodel.dist_x * (4.6 - kittmodel.dist_y)
product2 = (4.6 - kittmodel.dist_x) * kittmodel.dist_y
product3 = kittmodel.dist_x * kittmodel.dist_y
product4 = (4.6 - kittmodel.dist_x) * (4.6 - kittmodel.dist_y)

if kittmodel.obstacle_trajectory[0] >= 0 and kittmodel.obstacle_trajectory[1] >= 0:
    kittmodel.avoidance_turn = 100 if product1 >= product2 else 200
elif kittmodel.obstacle_trajectory[0] <= 0 and kittmodel.obstacle_trajectory[1] >= 0:
    kittmodel.avoidance_turn = 200 if product3 >= product4 else 100
elif kittmodel.obstacle_trajectory[0] >= 0 and kittmodel.obstacle_trajectory[1] <= 0:
    kittmodel.avoidance_turn = 200 if product3 >= product4 else 100
else:
    kittmodel.avoidance_turn = 200 if product1 >= product2 else 100
